## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Batsfjord,NO,70.63,29.72,32.07,87,99,21.32,light snow
1,1,Nome,US,64.50,-165.41,26.67,74,75,13.80,broken clouds
2,2,Flin Flon,CA,54.77,-101.86,44.44,65,75,14.97,broken clouds
3,3,Bluff,NZ,-46.60,168.33,54.72,86,76,7.34,broken clouds
4,4,Ushuaia,AR,-54.80,-68.30,35.26,87,40,20.71,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Vao,NC,-22.67,167.48,72.45,69,42,13.40,scattered clouds
6,6,Faanui,PF,-16.48,-151.75,75.13,89,98,7.94,moderate rain
11,11,Rikitea,PF,-23.12,-134.97,75.31,72,78,20.11,broken clouds
20,20,Saint George,US,37.10,-113.58,79.27,25,20,11.99,few clouds
25,25,Atuona,PF,-9.80,-139.03,77.50,75,25,16.04,light rain
26,26,Avarua,CK,-21.21,-159.78,77.05,61,75,14.97,broken clouds
29,29,Lazaro Cardenas,MX,17.96,-102.20,76.48,91,100,6.78,overcast clouds
46,46,Vaini,TO,-21.20,-175.20,78.96,78,20,10.36,few clouds
56,56,Guerrero Negro,MX,27.98,-114.06,70.02,65,17,16.98,few clouds
59,59,Odweyne,SO,9.41,45.06,73.54,48,75,2.93,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                170
City                   170
Country                170
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Current Description    170
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vao,NC,72.45,scattered clouds,-22.67,167.48,
6,Faanui,PF,75.13,moderate rain,-16.48,-151.75,
11,Rikitea,PF,75.31,broken clouds,-23.12,-134.97,
20,Saint George,US,79.27,few clouds,37.10,-113.58,
25,Atuona,PF,77.50,light rain,-9.80,-139.03,
26,Avarua,CK,77.05,broken clouds,-21.21,-159.78,
29,Lazaro Cardenas,MX,76.48,overcast clouds,17.96,-102.20,
46,Vaini,TO,78.96,few clouds,-21.20,-175.20,
56,Guerrero Negro,MX,70.02,few clouds,27.98,-114.06,
59,Odweyne,SO,73.54,broken clouds,9.41,45.06,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vao,NC,72.45,scattered clouds,-22.67,167.48,Hotel Kou-Bugny
6,Faanui,PF,75.13,moderate rain,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
11,Rikitea,PF,75.31,broken clouds,-23.12,-134.97,People ThankYou
20,Saint George,US,79.27,few clouds,37.10,-113.58,Best Western Plus Abbey Inn
25,Atuona,PF,77.50,light rain,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
678,Nakapiripirit,UG,70.18,broken clouds,1.92,34.78,
689,Gushikawa,JP,72.63,clear sky,26.35,127.87,Kirari Guest House
696,Sambava,MG,73.71,broken clouds,-14.27,50.17,Melrose
697,Caravelas,BR,75.06,overcast clouds,-17.71,-39.25,Pousada dos Navegantes


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))